In [ ]:
from typing import Iterable
import numpy as np

from view_sampler import *
from algs import *

import loss_funcs
from evaluate import eval_funcs
from eval_process import evaluate

from evaluate.dataset import Dataset
from evaluate.eval_log import EvalLog
from evaluate.evaluator import Evaluator
from evaluate.dataset import Dataset
from utils.concurrent import TqdmPool

import mealpy

import config
import concurrent
from itertools import product



In [ ]:

exec = TqdmPool(max_workers=30)

run_config = MealRunConfig(max_time=15, silent=True, seed=0)

# dataset = Dataset.create_random(location=config.OBJ_LOCATION, num_samples=20, seed=0)
# dataset = Dataset.load("eval_dataset.pkl")
# dataset.save("AllOptimizersDataset.pkl")

dataset = config.EVAL_DATASET

futures = []
optimizers = ['OriginalFFA','OriginalPSO', 'OriginalDE']

for idx, alg_name in enumerate(optimizers):
    for alg_config in product(*config.ALGORITHM_PARAMS[alg_name].values()):
        keys = config.ALGORITHM_PARAMS[alg_name].keys()
        alg_config = dict(zip(keys, alg_config))
        print(f"{alg_name}::{alg_config}")
        for obj_name in config.OBJECT_NAMES:
            future = exec.submit(
                evaluate,
                alg_name=alg_name,
                run_config=run_config,
                obj_name=obj_name,
                eval_data=config.EVAL_DATASET,
                log_folder=f"AlgTuning/Eval2/{obj_name}",
                alg_config=alg_config,
            )

            futures.append(future)

        
for future in concurrent.futures.as_completed(futures):
    try:
        future.result()
    except Exception as e:
        print(e)

exec.shutdown(wait=True)
